# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments             Customer  \
0  Aug  1 2022  9:32AM  243129        19  ADV80004571  AdvaGen Pharma, Ltd   
1  Aug  1 2022  9:28AM  243127        19  ADV80004568  AdvaGen Pharma, Ltd   
2  Aug  1 2022  9:26AM  243125        10   0085916953    ISDIN Corporation   
3  Aug  1 2022  9:26AM  243125        10   0085916980    ISDIN Corporation   
4  Aug  1 2022  9:26AM  243125        10   0085917014    ISDIN Corporation   
5  Aug  1 2022  9:26AM  243125        10   0085917015    ISDIN Corporation   
6  Aug  1 2022  9:26AM  243125        10   0085917017    ISDIN Corporation   
7  Aug  1 2022  9:26AM  243125        10   0085917016    ISDIN Corporation   
8  Aug  1 2022  9:26AM  243125        10   0085917021    ISDIN Corporation   
9  Aug  1 2022  9:26AM  243125        10   0085917027    ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
14  243123       Released          2
15  243124       Released          1
16  243125       Released         21
17  243127       Released          1
18  243129       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
243123                NaN       2.0
243124                NaN       1.0
243125                NaN      21.0
243127                NaN       1.0
243129                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
243100                7.0       0.0
243108                0.0      32.0
243109                1.0       2.0
243110                0.0       3.0
243113                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
243100                  7         0
243108                  0        32
243109                  1         2
243110                  0         3
243113                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               243100          7         0
1               243108          0        32
2               243109          1         2
3               243110          0         3
4               243113          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               243100         7         
1               243108                 32
2               243109         1        2
3               243110                  3
4               243113                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Aug  1 2022  9:32AM  243129        19             AdvaGen Pharma, Ltd
1   Aug  1 2022  9:28AM  243127        19             AdvaGen Pharma, Ltd
2   Aug  1 2022  9:26AM  243125        10               ISDIN Corporation
23  Aug  1 2022  9:14AM  243124        19                  Graystone, LLC
24  Aug  1 2022  9:13AM  243123        16  Sartorius Bioprocess Solutions
26  Aug  1 2022  9:11AM  243122        12                       Hush Hush
71  Aug  1 2022  8:54AM  243121        10       Beach Patient Assistance 
72  Aug  1 2022  8:30AM  243117        16            Emersa Waterbox, LLC
73  Aug  1 2022  8:30AM  243100        10               ISDIN Corporation
80  Aug  1 2022  8:29AM  243118        10    Snap Medical Industries, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Aug  1 2022  9:32AM  243129        19             AdvaGen Pharma, Ltd   
1  Aug  1 2022  9:28AM  243127        19             AdvaGen Pharma, Ltd   
2  Aug  1 2022  9:26AM  243125        10               ISDIN Corporation   
3  Aug  1 2022  9:14AM  243124        19                  Graystone, LLC   
4  Aug  1 2022  9:13AM  243123        16  Sartorius Bioprocess Solutions   
5  Aug  1 2022  9:11AM  243122        12                       Hush Hush   
6  Aug  1 2022  8:54AM  243121        10       Beach Patient Assistance    
7  Aug  1 2022  8:30AM  243117        16            Emersa Waterbox, LLC   
8  Aug  1 2022  8:30AM  243100        10               ISDIN Corporation   
9  Aug  1 2022  8:29AM  243118        10    Snap Medical Industries, LLC   

  Executing Released  
0                  1  
1                  1  
2                 21  
3                  1  
4                  2  
5         4       41  
6                  1  
7                  1  
8         7           
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Aug  1 2022  9:32AM  243129        19             AdvaGen Pharma, Ltd   
1  Aug  1 2022  9:28AM  243127        19             AdvaGen Pharma, Ltd   
2  Aug  1 2022  9:26AM  243125        10               ISDIN Corporation   
3  Aug  1 2022  9:14AM  243124        19                  Graystone, LLC   
4  Aug  1 2022  9:13AM  243123        16  Sartorius Bioprocess Solutions   
5  Aug  1 2022  9:11AM  243122        12                       Hush Hush   
6  Aug  1 2022  8:54AM  243121        10       Beach Patient Assistance    
7  Aug  1 2022  8:30AM  243117        16            Emersa Waterbox, LLC   
8  Aug  1 2022  8:30AM  243100        10               ISDIN Corporation   
9  Aug  1 2022  8:29AM  243118        10    Snap Medical Industries, LLC   

  Released Executing  
0        1            
1        1            
2       21            
3        1            
4        2            
5       41         4  
6        1            
7        1            
8                  7  
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Aug  1 2022  9:32AM  243129        19             AdvaGen Pharma, Ltd   
1  Aug  1 2022  9:28AM  243127        19             AdvaGen Pharma, Ltd   
2  Aug  1 2022  9:26AM  243125        10               ISDIN Corporation   
3  Aug  1 2022  9:14AM  243124        19                  Graystone, LLC   
4  Aug  1 2022  9:13AM  243123        16  Sartorius Bioprocess Solutions   

  Released Executing  
0        1            
1        1            
2       21            
3        1            
4        2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                        Customer  \
0  Aug  1 2022  9:32AM  243129        19             AdvaGen Pharma, Ltd   
1  Aug  1 2022  9:28AM  243127        19             AdvaGen Pharma, Ltd   
2  Aug  1 2022  9:26AM  243125        10               ISDIN Corporation   
3  Aug  1 2022  9:14AM  243124        19                  Graystone, LLC   
4  Aug  1 2022  9:13AM  243123        16  Sartorius Bioprocess Solutions   

   Released  Executing  
0       1.0        NaN  
1       1.0        NaN  
2      21.0        NaN  
3       1.0        NaN  
4       2.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Aug  1 2022  9:32AM  243129        19             AdvaGen Pharma, Ltd   
1  Aug  1 2022  9:28AM  243127        19             AdvaGen Pharma, Ltd   
2  Aug  1 2022  9:26AM  243125        10               ISDIN Corporation   
3  Aug  1 2022  9:14AM  243124        19                  Graystone, LLC   
4  Aug  1 2022  9:13AM  243123        16  Sartorius Bioprocess Solutions   

   Released  Executing  
0       1.0        0.0  
1       1.0        0.0  
2      21.0        0.0  
3       1.0        0.0  
4       2.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         1944919      37.0        7.0
12          243122      41.0        4.0
16          486240       3.0        0.0
19         1458710      38.0        1.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  1944919      37.0        7.0
1        12   243122      41.0        4.0
2        16   486240       3.0        0.0
3        19  1458710      38.0        1.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      37.0        7.0
1        12      41.0        4.0
2        16       3.0        0.0
3        19      38.0        1.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released   37.0
1        12   Released   41.0
2        16   Released    3.0
3        19   Released   38.0
4        10  Executing    7.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12   16    19
Status                          
Executing   7.0   4.0  0.0   1.0
Released   37.0  41.0  3.0  38.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12   16    19
0          Executing   7.0   4.0  0.0   1.0
1           Released  37.0  41.0  3.0  38.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12   16    19
0  Executing   7.0   4.0  0.0   1.0
1   Released  37.0  41.0  3.0  38.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()